# PDF to images and text

## Imports

In [4]:
# pip install PyMuPDF

In [1]:
import re
import fitz
import json
from pathlib import Path
from tqdm.auto import tqdm

from castom_utils import load_config, write_json

In [2]:
CFG = load_config()

Config is loaded!
List of all chars used (114):
0123456789!"'#%&()*+,-./:;<=>?@[\]^_{|}~№°$£€¥¢₽АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя


## Convert

In [9]:
def pdf_to_images_and_text(
    pdf_dir: Path, 
    output_dir: Path, 
    dpi: int = 300
):
    """
    Нарезка PDF файлов с текстовым слоем на отдельные изображения и файл с текстами.
    """
    output_dir.joinpath('images').mkdir(parents=True, exist_ok=True)
    annotations = {}

    # Получаем список всех *.pdf документов
    pdf_paths = pdf_dir.glob('*.pdf')
    
    for pdf_path in pdf_paths:
        pdf_name = pdf_path.name
        pdf_document = fitz.open(pdf_path)
        total_pages = pdf_document.page_count

        # Обработка каждой страницы
        for page_num in tqdm(range(total_pages), desc=pdf_name):
            page = pdf_document.load_page(page_num)

            # Сохраняем изображение
            pix = page.get_pixmap(dpi=dpi)
            img_name = f"{pdf_name.rstrip('.pdf')}_page{page_num + 1:04d}.png"
            img_path = output_dir.joinpath('images', img_name)
            pix.save(img_path)

            # Сохраняем текстовый слой
            text = page.get_text()
            
            # Очистка текста
            text = re.sub(r' \n', '\n', text)
            text = re.sub(r'\xad', '-', text)
            text = re.sub(r'[^\S\n]+', ' ', text)
            # text = re.sub(r'[\n\t\r]+', '', text)

            annotations[img_name] = {'text': text}

    write_json(output_dir / 'labels.json', annotations)

In [10]:
pdf_dir = CFG.pdf_files
output_dir = CFG.dataset_documents

pdf_to_images_and_text(pdf_dir, output_dir)

zadachi.pdf:   0%|          | 0/21 [00:00<?, ?it/s]

book.pdf:   0%|          | 0/108 [00:00<?, ?it/s]

In [ ]:
write_json()